# Mise en oeuvre du modèle Mistral Large (123B paramètres ?)

Julien Velcin, Université Lyon 2 - Master Humanités Numériques

Plus de détails sur le modèle : https://mistral.ai/fr/news/mistral-large/

La partie RAG suit le traitement proposé ici : https://docs.mistral.ai/guides/rag/

Il est possible d'affiner (fine tune) ce type de modèle, y compris via l'API si on n'a pas assez de ressources de calcul en local, mais ce ne sera pas vu dans ce cours : https://docs.mistral.ai/guides/finetuning/

Ce qui suit est une démonstration de l'utilisation de l'API de Mistral, comme alternative aux modèles de référence comme GPT. A noter qu'il est possible, en plus de faire de la génération à partir de prompt et des données de préentrainement du modèle, de suivre le principe RAG (Retrieval Augmented Generation) afin de coupler ça avec une base de données qui a été indexée.

In [61]:
# librairies utilisées

import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import time
import faiss
import pickle
import faiss.contrib.torch_utils

Tout d'abord, il s'agit de charger la clef d'accès à l'API de Mistral (gratuit pour une utilisation individuelle mais potentiellement limitée).

In [ ]:
api_key = os.environ["MISTRAL_KEY_API"]

Après avoir [installé la librairie *mistralai*](https://docs.mistral.ai/getting-started/clients/), vous pouvez la charger en mémoire et choisir votre modèle.

In [2]:
from mistralai import Mistral

# choix du modèle (ici la version large)
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

Pour commencer, quelques tests simples d'autocomplétion, en anglais...

In [27]:
chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user", # "rôle" que va jouer le robot (liste à choix fermée)
            "content": "What is the best French cheese?", # la question
        },
    ]
)
print(chat_response.choices[0].message.content)

Choosing the "best" French cheese can be subjective, as it depends on personal taste. France is renowned for its wide variety of cheeses, with over 400 different types. Here are a few highly regarded French cheeses across various categories:

1. **Soft Cheeses**:
   - **Brie de Meaux**: Known for its creamy texture and rich, slightly nutty flavor.
   - **Camembert de Normandie**: Soft and creamy with a strong, earthy aroma.

2. **Semi-Soft Cheeses**:
   - **Morbier**: Recognizable by its layer of ash in the middle, it has a fruity and slightly nutty taste.
   - **Reblochon**: A nutty and fruity cheese from the Alps, often used in tartiflette.

3. **Hard Cheeses**:
   - **Comté**: A firm, nutty, and slightly sweet cheese made from unpasteurized cow's milk.
   - **Beaufort**: Similar to Comté, it has a firm texture and a complex, nutty flavor.

4. **Blue Cheeses**:
   - **Roquefort**: A tangy and salty sheep milk cheese with distinctive veins of blue mold.
   - **Bleu d'Auvergne**: A str

...et en français :

In [3]:
chat_response_fr = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "La réponse s'adresse à un locuteur français. Quel est le meilleur fromage ?",
        },
    ]
)
print(chat_response_fr.choices[0].message.content)

La question du "meilleur fromage" est très subjective et dépend des goûts personnels de chacun. La France, par exemple, est célèbre pour sa grande variété de fromages, chacun ayant ses propres caractéristiques et saveurs. Voici quelques fromages français très appréciés :

1. **Camembert** : Un fromage à pâte molle et à croûte fleurie, originaire de Normandie.
2. **Roquefort** : Un fromage persillé (à pâte bleue) fabriqué à partir de lait de brebis, originaire de l'Aveyron.
3. **Brie de Meaux** : Un fromage à pâte molle et à croûte fleurie, souvent considéré comme le "roi des fromages".
4. **Comté** : Un fromage à pâte pressée cuite, originaire du Jura et de la Franche-Comté.
5. **Reblochon** : Un fromage à pâte pressée non cuite, originaire de la Savoie et de la Haute-Savoie.
6. **Chèvre** : Il existe de nombreuses variétés de fromages de chèvre, comme le Crottin de Chavignol ou le Sainte-Maure de Touraine.

Chacun de ces fromages a ses propres caractéristiques et peut être apprécié de

On peut aussi demander à un modèle proche de fournir les plongements (*embeddings*) de textes :

In [30]:
model_emb = "mistral-embed"

client = Mistral(api_key=api_key)

embeddings_response = client.embeddings.create(
    model=model_emb,
    inputs=["Un fromage à pâte pressée cuite, originaire du Jura et de la Franche-Comté.",
            "Il existe de nombreuses variétés de fromages de chèvre, comme le Crottin de Chavignol ou le Sainte-Maure de Tourain.",
           "Le pithiviers est un gâteau français à base de pâte feuilletée originaire de la commune de Pithiviers située dans le département du Loiret et la région Centre-Val de Loire.",
           "Le château de Versailles est un château et un monument historique français situé à Versailles, dans les Yvelines."]
)

In [31]:
vectors = [np.array(e.embedding) for e in embeddings_response.data]
print(len(vectors))

4


In [54]:
# similarité cosinus entre deux vecteurs sous numpy
def cosine(x, y):
    return float(cosine_similarity(x.reshape(1, -1), y.reshape(1, -1)).squeeze())

In [55]:
print(f"Similarité cosinus : {cosine(vectors[0], vectors[1]):.3f} ")

Similarité cosinus : 0.830 


# Combiner Mistral avec du RAG

Le RAG (*Retrieval Augmented Generation*) permet de combiner la recherche d'information (pour trouver les meilleurs passages) et la génération d'une réponse.

La solution présentée ici ne recourt pas à des librairies comme Langchain ou LlamaIndex, que nous ne verrons pas en cours, mais beaucoup de ressources en ligne en parlent très bien (comme : https://docs.mistral.ai/guides/rag/).

Chargement des données textuelles à partir d'un fichier (cf. début du cours) :

In [4]:
with open(os.path.join("data", "alice.txt")) as f:    
    lines = [line.strip() for line in f.readlines()]

print(f"{len(lines)} lignes dans le fichier texte")

5777 lignes dans le fichier texte


On combine les lignes pour produire un corpus de paragraphes (il faut éviter les documents vides).

In [5]:
docs = []
s = ""
for l in lines:
    if (l != ""):
        s = s + " " + l
    else:
        if (s != ""):
            docs = docs + [s]
            s = ""

print(f"{len(docs)} paragraphes")

2033 paragraphes


On choisit le modèle d'embedding et on initialise le modèle (comme tout à l'heure).

In [12]:
model_emb = "mistral-embed"

client = Mistral(api_key=api_key)

Si la base de données vectorielle n'a pas été créée, il faut le faire (cela peut prendre du temps). Attribuer alors valeur *True* à la variable *build_embeddings* pour lancer le travail avec l'API.

In [90]:
build_embeddings = False # mettre True pour recalculer tous les plongements

if build_embeddings:

    text_embeddings = []
    docs_2processed = docs.copy()
    
    step = 512 # nombre de documents traités à la fois (à priori, max 512 pour Mistral large)

    while (len(docs_2processed)>step):
        # liste des documents à traiter
        liste_docs = docs_2processed[:step]

        #on demande à l'API de calculer les vecteurs
        embeddings_response = client.embeddings.create(
            model=model_emb,
            inputs = liste_docs
        )

        # on ajoute à la liste des vecteurs déjà calculés les nouveaux plongeements
        text_embeddings.append([np.array(e.embedding) for e in embeddings_response.data])

        # on décale le "curseur" sur la liste de traitement
        docs_2processed = docs_2processed[step:]
        print(len(liste_docs))

        # petit temps de latence pour ne pas surcharger l'API
        time.sleep(5)

    # même traitement mais pour les derniers documents (liste de longueur < step)
    if (len(docs_2processed)>0):
        liste_docs = docs_2processed
        embeddings_response = client.embeddings.create(
            model=model_emb,
            inputs = liste_docs
        )
        print(len(liste_docs))
        text_embeddings.append([np.array(e.embedding) for e in embeddings_response.data])

    # On construit une liste de tableau format numpy à partir de la liste des tableaux (c'est une simple concaténation)
    vectors_list = np.array([j for i in text_embeddings for j in i] )

    # On pense à sauvegarder le tableau de plongement dans un fichier sur le disque
    with open("save_emb_alice.pkl", "wb") as f:
        pickle.dump(vectors_list, f)

512
512
512
497


In [7]:
if not build_embeddings:
with open("save_emb_alice.pkl", "rb") as f:
    vectors_list = pickle.load(f)

In [8]:
vectors_list.shape

(2033, 1024)

A partir de là, on peut créer l'index de la base de données vectorielle grâce à la libaririe *faiss* :

In [56]:
d = vectors_list_f32.shape[1]
print(d)
index = faiss.IndexFlatL2(d)
print(index.is_trained)
index.add(vectors_list_f32)

1024
True


Note : pour accélérer les choses, le mieux serait de sauvegarder cet index sur le disque et pas de le recalculer à chaque fois.

# Poser des questions sur les données textuelles

A présent, on peut utiliser l'index pour poser n'importe quelle question.

In [65]:
def poser_une_question(question):

    # création du plongement (=vecteur) correspondant à la question posée
    embeddings_response = client.embeddings.create(
        model=model_emb,
        inputs = question
    )
    question_embedding = np.array([embeddings_response.data[0].embedding])

    # recherche d'information : on sélectionne les 2 documents ayant des vecteurs les plus similaires à la question (cf. TD1)
    D, I = index.search(question_embedding, k=2)
    retrieved_docs = [docs[i] for i in I.tolist()[0]]

    # on crée l'invite (prompt) qui va nous permettre d'interroger le modèle de langue
    prompt = f"""
        Context information is below.
        ---------------------
            {retrieved_docs}
        ---------------------
        Given the context information and not prior knowledge, answer the query.
            Query: {question}
        Answer:
        """
    messages = [
        {
            "role": "user", "content": prompt
        }
    ]
    # ajout d'un petit temps de latence pour éviter de surcharger l'API
    time.sleep(5)

    # on lance la requête en ligne
    chat_response = client.chat.complete(
        model="mistral-large-latest",
        messages=messages
    )
    return (chat_response.choices[0].message.content) 

In [66]:
question1 = "What unexpected animal the duchess comes out carrying in baby's clothes?"
question2 = "What is the usual fate reserved for the queen's enemies?"

print(poser_une_question(question1))

The Duchess comes out carrying a pig in baby's clothes.


In [67]:
print(poser_une_question(question2))

The usual fate reserved for the queen's enemies is to be beheaded.


Les réponses apportée semblent tout à fait pertinentes.